In [1]:
!pkill -f "Microsoft Excel"

In [2]:
import glob
import xlwings as xw
import pandas as pd
import itertools
import copy

In [34]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./dev/리스/리스 데이터정리.xlsx')

In [35]:
data = wb.sheets['현대'].range('B1:AA2000').value
df = pd.DataFrame(data[1:], columns=data[0])
df = df.dropna(subset = 'brand_name')
df = df[df['판매여부'] != 'N']

In [36]:
wb.close()

In [41]:
print(df.columns)
df.head(3)

Index(['brand_name', 'brand_img', 'car_name', 'model_name', 'model_detial',
       'car_price', 'ext_color', 'int_color', 'is_limited', '판매여부', 'brand_im',
       'car_im', 'brand_jb', 'car_jb', 'brand_lotte', 'car_lotte',
       'model_lotte', 'brand_mz', 'car_mz', 'brand_nh', 'car_nh', 'brand_sh',
       'car_sh', 'brand_woori', 'car_woori', '전기차 보조금'],
      dtype='object')


,brand_name,brand_img,car_name,model_name,model_detial,car_price,ext_color,int_color,is_limited,판매여부,...,model_lotte,brand_mz,car_mz,brand_nh,car_nh,brand_sh,car_sh,brand_woori,car_woori,전기차 보조금
0,현대,https://www.carpan.co.kr/img/brand/1001.png,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),스마트,14600000.0,"[['언블리치드 아이보리(NES)', 'background-color: rgb(18...","[['블랙', 'background-color: rgb(0, 0, 0);']]",None,None,...,더 뉴캐스퍼 가솔린 1.0 스마트,1.0,2.0,2.0,61.0,2.0,79.0,None,None,None
1,현대,https://www.carpan.co.kr/img/brand/1001.png,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),디 에센셜,16800000.0,"[['선택 취소', 'background-color: rgb(182, 179, 17...","[['블랙', 'background-color: rgb(0, 0, 0);'], ['...",None,None,...,더 뉴캐스퍼 가솔린 1.0 디 에센셜,1.0,2.0,2.0,61.0,2.0,79.0,None,None,None
2,현대,https://www.carpan.co.kr/img/brand/1001.png,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),인스퍼레이션,19800000.0,"[['언블리치드 아이보리(NES)', 'background-color: rgb(18...","[['블랙', 'background-color: rgb(0, 0, 0);'], ['...",None,None,...,더 뉴캐스퍼 가솔린 1.0 인스퍼레이션,1.0,2.0,2.0,61.0,2.0,79.0,None,None,None


In [38]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./dev/리스/견적기/bnk.xlsm')

In [39]:
def brand_idx(sh, x):
    x = x.strip().replace('\xa0', ' ')
    models = sh.range('J7', 'J36').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_idx(sh,x):
    x = x.strip().replace('\xa0', ' ')
    models = sh.range('AM20', 'AM99').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_model_idx(sh,x):
    x = str(x).strip().replace('\xa0', ' ')
    models = sh.range('AO20', 'AO256').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1
    
def convert_params( brand_name, car_name, model_name, car_price, org_brand_name, org_car_name, org_model_name):
    input_data = {
        "bnk": {
            'brand_name': [0, 'B9', brand_name],
            'car_name': [0, 'B13', car_name],
            'model_name': [0, 'B15', model_name],
            'car_price': [1, 'N13', car_price],
            'deposit': [1, 'N36', 0],
            'downpayment': [1, 'N38', 0],
            'period': [0, 'B39', 0],
            'org_brand_name': org_brand_name,
            'org_car_name': org_car_name,
            'org_model_name': org_model_name,
        },
    }
    return input_data  # 디버깅을 위해 input_data 출력


In [40]:
sheet_names = ['Es1', 'Sheet1']
sheets = [wb.sheets[i] for i in sheet_names]

input_datum = df.apply(lambda x: convert_params(x['brand_BNK'], x['car_BNK'], x['model_BNK'], x['car_price'], x['brand_name'], x['car_name'], x['model_name']), axis=1)

KeyError: 'brand_BNK'

In [8]:
deposits = [0.1, 0.2, 0.3, 0.4, 0.5]
downpayments = [0.1, 0.2, 0.3, 0.4, 0.5] 
periods = [1, 2, 3] # 60, 48, 36 

output = []

for idx, input_data in enumerate(input_datum):
    inputs = []  
    for combo in itertools.product(deposits, downpayments, periods):
        modified_input = copy.deepcopy(input_data)
        modified_input['bnk']['deposit'][-1] = combo[0]
        modified_input['bnk']['downpayment'][-1] = combo[1]
        modified_input['bnk']['period'][-1] = combo[2]
        inputs.append(modified_input)
    for input_data in inputs:
        for i in input_data['bnk'].items():
            if 'org' not in i[0]:
                val = i[1][2]
                if i[0] == 'brand_name':
                    d = brand_idx(sheets[0], val)
                elif i[0] == 'car_name':
                    d = car_idx(sheets[0], val)
                elif i[0] == 'model_name':
                    d = car_model_idx(sheets[0], val)        
                else:
                    d = i[1][2]
                sheets[i[1][0]].range(i[1][1]).value = d 
        output.append(
            {
                "input" : input_data,
                "output": sheets[1].range('N51').value
            }
        )
    if idx==2:
        break


KeyboardInterrupt



In [10]:
records = []
for entry in output:
    input_data = entry['input']['bnk']
    output_value = entry['output']
    record = {
        'carpan_brand_name': input_data['org_brand_name'].strip().replace('\xa0', ' '),
        'carpan_car_name': input_data['org_car_name'].strip().replace('\xa0', ' '),
        'carpan_model_name': input_data['org_model_name'].strip().replace('\xa0', ' '),
        'provider' : 'bnk',
        'brand_name': input_data['brand_name'][2].strip().replace('\xa0', ' '),
        'car_name': input_data['car_name'][2].strip().replace('\xa0', ' '),
        'model_name': input_data['model_name'][2].strip().replace('\xa0', ' '),
        'car_price': input_data['car_price'][2],
        'deposit': input_data['deposit'][2],
        'downpayment': input_data['downpayment'][2],
        'period': input_data['period'][2],
        'output': output_value
    }
    records.append(record)
# DataFrame 생성
df = pd.DataFrame(records)

In [11]:
def convert_period(x):
    if x == 1:
        return '60개월'
    if x == 2:
        return '48개월'
    if x == 3:
        return '36개월'
    
df['period'] = df['period'].apply(lambda x: convert_period(x))

In [12]:
df.to_csv('./bnk_현대_캐스퍼.csv', encoding = 'cp949') 

In [9]:
wb.close()

array(['2025년형 가솔린 1.0\n(2024.10~)'], dtype=object)